# Preprocesamiento de datos

In [1]:
import pandas as pd
import openpyxl 
import numpy as np
import os
from pathlib import Path
import unicodedata
from pyspark.sql.functions import desc
from pyspark.sql.functions import count, countDistinct
from pyspark.sql.functions import col

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PreprocesamientoINE").getOrCreate()

25/10/08 16:51:35 WARN Utils: Your hostname, Arielas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.122 instead (on interface en0)
25/10/08 16:51:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 16:51:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Funciones

In [3]:
def limpiar_nombre_columna(col, seen):
    col_clean = ''.join(
        c for c in unicodedata.normalize('NFD', str(col))
        if unicodedata.category(c) != 'Mn'
    )
    col_clean = (
        col_clean.strip()
        .lower()
        .replace(" ", "_")
        .replace(".", "_")
        .replace("-", "_")
    )

    if col_clean in seen:
        seen[col_clean] += 1
        col_clean = f"{col_clean}_{seen[col_clean]}"
    else:
        seen[col_clean] = 0
    return col_clean

def unir_excels_y_guardar_tabla(carpeta, nombre_tabla):
    folder_path = Path(carpeta)
    all_data = []

    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".xlsx"):
            year = int(''.join(filter(str.isdigit, file)))
            df = pd.read_excel(folder_path / file)
            df["año"] = year
            all_data.append(df)

    df_all = pd.concat(all_data, ignore_index=True)

    seen = {}
    df_all.columns = [limpiar_nombre_columna(c, seen) for c in df_all.columns]
    df_all = df_all.loc[:, ~df_all.columns.duplicated()]

    for col in ["año", "mes_ocu", "hora_ocu", "edad_per"]:
        if col in df_all.columns:
            df_all[col] = pd.to_numeric(df_all[col], errors="coerce").astype("Int64")

    for col in df_all.columns:
        if df_all[col].dtype == "object":
            df_all[col] = df_all[col].astype(str)

    df_spark = spark.createDataFrame(df_all)
    df_spark.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(nombre_tabla)

    print(f"Guardada tabla '{nombre_tabla}' con {df_spark.count()} filas")
    df_spark.printSchema()

## Carga de datos

In [223]:

dic_fallecidos = pd.read_excel('ine/Diccionario Fallecidos y Lesionados.xlsx', header=1)
dic_hechos = pd.read_excel('ine/Diccionario Hechos.xlsx', header=1)
dic_vehiculos = pd.read_excel('ine/Diccionario Vehículos.xlsx', header= 1)

# Fallecidos
fallecidos15 = pd.read_excel('ine/Fallecidos/fallecidos2015.xlsx')
fallecidos16 = pd.read_excel('ine/Fallecidos/fallecidos2016.xlsx')
fallecidos17 = pd.read_excel('ine/Fallecidos/fallecidos2017.xlsx')
fallecidos18 = pd.read_excel('ine/Fallecidos/fallecidos2018.xlsx')
fallecidos19 = pd.read_excel('ine/Fallecidos/fallecidos2019.xlsx')
fallecidos20 = pd.read_excel('ine/Fallecidos/fallecidos2020.xlsx')
fallecidos21 = pd.read_excel('ine/Fallecidos/fallecidos2021.xlsx')
fallecidos22 = pd.read_excel('ine/Fallecidos/fallecidos2022.xlsx')
fallecidos23 = pd.read_excel('ine/Fallecidos/fallecidos2023.xlsx')

lista_fallecidos = [fallecidos15, fallecidos16, fallecidos17, fallecidos18, fallecidos19, fallecidos20, fallecidos21, fallecidos22, fallecidos23]

# Hechos
hechos15 = pd.read_excel('ine/Hechos/hechos2015.xlsx')
hechos16 = pd.read_excel('ine/Hechos/hechos2016.xlsx')
hechos17 = pd.read_excel('ine/Hechos/hechos2017.xlsx')
hechos18 = pd.read_excel('ine/Hechos/hechos2018.xlsx')
hechos19 = pd.read_excel('ine/Hechos/hechos2019.xlsx')
hechos20 = pd.read_excel('ine/Hechos/hechos2020.xlsx')
hechos21 = pd.read_excel('ine/Hechos/hechos2021.xlsx')
hechos22 = pd.read_excel('ine/Hechos/hechos2022.xlsx')
hechos23 = pd.read_excel('ine/Hechos/hechos2023.xlsx')

lista_hechos = [hechos15, hechos16, hechos17, hechos18, hechos19, hechos20, hechos21, hechos22, hechos23]

# Hechos
vehiculos15 = pd.read_excel('ine/Vehiculos/vehiculos2015.xlsx')
vehiculos16 = pd.read_excel('ine/Vehiculos/vehiculos2016.xlsx')
vehiculos17 = pd.read_excel('ine/Vehiculos/vehiculos2017.xlsx')
vehiculos18 = pd.read_excel('ine/Vehiculos/vehiculos2018.xlsx')
vehiculos19 = pd.read_excel('ine/Vehiculos/vehiculos2019.xlsx')
vehiculos20 = pd.read_excel('ine/Vehiculos/vehiculos2020.xlsx')
vehiculos21 = pd.read_excel('ine/Vehiculos/vehiculos2021.xlsx')
vehiculos22 = pd.read_excel('ine/Vehiculos/vehiculos2022.xlsx')
vehiculos23 = pd.read_excel('ine/Vehiculos/vehiculos2023.xlsx')

lista_vehiculos = [vehiculos15, vehiculos16, vehiculos17, vehiculos18, vehiculos19, vehiculos20, vehiculos21, vehiculos22, vehiculos23]


## Diccionarios

In [5]:

diccionario_fallecidos = {}
variable_actual = None

for _, row in dic_fallecidos.iterrows():
    if pd.notna(row["Variable"]):  
        variable_actual = row["Variable"]
        diccionario_fallecidos[variable_actual] = {}
    
    # Intentar convertir a número
    codigo = pd.to_numeric(row["Código"], errors="coerce")
    valor = row["Valor"]

    if variable_actual and pd.notna(codigo) and pd.notna(valor):
        diccionario_fallecidos[variable_actual][int(codigo)] = str(valor)

diccionario_fallecidos = {var: {v: k for k, v in mapa.items()} for var, mapa in diccionario_fallecidos.items()}

In [6]:

diccionario_hechos = {}
variable_actual = None

for _, row in dic_hechos.iterrows():
    if pd.notna(row["Variable"]):  
        variable_actual = row["Variable"]
        diccionario_hechos[variable_actual] = {}
    
    # Intentar convertir a número
    codigo = pd.to_numeric(row["Código"], errors="coerce")
    valor = row["Valor"]

    if variable_actual and pd.notna(codigo) and pd.notna(valor):
        diccionario_hechos[variable_actual][int(codigo)] = str(valor)

diccionario_hechos = {var: {v: k for k, v in mapa.items()} for var, mapa in diccionario_hechos.items()}
diccionario_hechos["g_hora_5"]

{'Mañana': 1, 'Tarde': 2, 'Noche': 3, 'Ignorada': 4}

In [7]:

diccionario_vehiculos = {}
variable_actual = None

for _, row in dic_vehiculos.iterrows():
    if pd.notna(row["Variable"]):  
        variable_actual = row["Variable"]
        diccionario_vehiculos[variable_actual] = {}
    
    # Intentar convertir a número
    codigo = pd.to_numeric(row["Código"], errors="coerce")
    valor = row["Valor"]

    if variable_actual and pd.notna(codigo) and pd.notna(valor):
        diccionario_vehiculos[variable_actual][int(codigo)] = str(valor)

diccionario_vehiculos = {var: {v: k for k, v in mapa.items()} for var, mapa in diccionario_vehiculos.items()}
diccionario_vehiculos["g_hora_5"]

{'Mañana': 1, 'Tarde': 2, 'Noche': 3, 'Ignorada': 4}

## Limpieza

### Fallecidos

In [8]:
diccionario_fallecidos['día_sem_ocu']['Sabado'] = 6
diccionario_fallecidos['día_sem_ocu']['Miercoles'] = 3
diccionario_fallecidos['día_sem_ocu']

{'Lunes': 1,
 'Martes': 2,
 'Miércoles': 3,
 'Jueves': 4,
 'Viernes': 5,
 'Sábado': 6,
 'Domingo': 7,
 'Sabado': 6,
 'Miercoles': 3}

In [9]:
for k in list(diccionario_fallecidos['mupio_ocu'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_fallecidos['mupio_ocu']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_fallecidos['mupio_ocu'][nk] = diccionario_fallecidos['mupio_ocu'].pop(k)

diccionario_fallecidos['mupio_ocu']

{'San José Pinula': 103,
 'San José del Golfo': 104,
 'Palencia': 105,
 'Chinautla': 106,
 'San Pedro Ayampuc': 107,
 'Mixco': 108,
 'San Pedro Sacatepéquez': 1202,
 'San Juan Sacatepéquez': 110,
 'San Raymundo': 111,
 'Chuarrancho': 112,
 'Fraijanes': 113,
 'Amatitlán': 114,
 'Villa Nueva': 115,
 'Villa Canales': 116,
 'Guastatoya': 201,
 'Morazán': 202,
 'San Agustín Acasaguastlán': 203,
 'El Jícaro': 205,
 'Sansare': 206,
 'Sanarate': 207,
 'San Antonio la Paz': 208,
 'Antigua Guatemala': 301,
 'Jocotenango': 302,
 'Pastores': 303,
 'Sumpango': 304,
 'Santo Domingo Xenacoj': 305,
 'Santiago Sacatepéquez': 306,
 'San Bartolomé Milpas Altas': 307,
 'San Lucas Sacatepéquez': 308,
 'Santa Lucía Milpas Altas': 309,
 'Magdalena Milpas Altas': 310,
 'Santa María de Jesús': 311,
 'Ciudad Vieja': 312,
 'San Miguel Dueñas': 313,
 'San Juan Alotenango': 314,
 'San Antonio Aguas Calientes': 315,
 'Santa Catarina Barahona': 316,
 'Chimaltenango': 401,
 'San José  Poaquíl': 402,
 'San Martín Jilo

In [10]:
diccionario_fallecidos['zona_ocu']['Ignorado'] = 99
diccionario_fallecidos['zona_ocu']

{'Ignorada': 99, 'Ignorado': 99}

In [27]:

for k in list(diccionario_fallecidos['edad_quinquenales'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_fallecidos['edad_quinquenales']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_fallecidos['edad_quinquenales'][nk] = diccionario_fallecidos['edad_quinquenales'].pop(k)

diccionario_fallecidos['edad_quinquenales']['80 o más'] = 17
diccionario_fallecidos['edad_quinquenales']['80 o m?s'] = 17
diccionario_fallecidos['edad_quinquenales']['70 - 75'] = 15
diccionario_fallecidos['edad_quinquenales']

{'0 - 4': 1,
 '5 - 9': 2,
 '10 - 14': 3,
 '15 - 19': 4,
 '20 - 24': 5,
 '25 - 29': 6,
 '30 - 34': 7,
 '35 - 39': 8,
 '40 - 44': 9,
 '45 - 49': 10,
 '50 - 54': 11,
 '55 - 59': 12,
 '60 - 64': 13,
 '65 - 69': 14,
 '70 - 74': 15,
 '75 - 79': 16,
 '80 y  más': 17,
 'Ignorado': 18,
 '80 o más': 17,
 '80 o m?s': 17,
 '70 - 75': 15}

In [12]:
diccionario_fallecidos['tipo_veh']['Camioneta'] = 25
diccionario_fallecidos['tipo_veh']

{'Automóvil': 1,
 'Camioneta sport o blazer': 2,
 'Pick up': 3,
 'Motocicleta': 4,
 'Camión': 5,
 'Cabezal': 6,
 'Bus extraurbano': 7,
 'Jeep': 8,
 'Microbús': 9,
 'Taxi': 10,
 'Panel': 11,
 'Bus urbano': 12,
 'Tractor': 13,
 'Moto taxi': 14,
 'Furgón': 15,
 'Grúa': 16,
 'Bus escolar': 17,
 'Bicicleta': 18,
 'Avioneta': 19,
 'Montacargas': 20,
 'Bus militar': 21,
 'Cuatrimoto': 22,
 'Furgoneta': 23,
 'Motos acuáticas': 24,
 'Ignorado': 99,
 'Camioneta': 25}

In [13]:
diccionario_fallecidos['tipo_eve']['Embarrancó'] = 7
diccionario_fallecidos['tipo_eve']

{'Colisión': 1,
 'Choque': 2,
 'Vuelco': 3,
 'Caída': 4,
 'Atropello': 5,
 'Derrape': 6,
 'Embarranco': 7,
 'Encuneto': 8,
 'Ignorado': 99,
 'Embarrancó': 7}

In [20]:
fallecidos22.replace(diccionario_fallecidos, inplace=True)
fallecidos23.replace(diccionario_fallecidos, inplace=True)

/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_36586/2898028915.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fallecidos22.replace(diccionario_fallecidos, inplace=True)


In [21]:
fallecidos_junto = pd.concat(lista_fallecidos)

In [22]:
fallecidos_junto.tail()

,núm_corre,año_ocu,mes_ocu,día_ocu,día_sem_ocu,hora_ocu,g_hora,g_hora_5,depto_ocu,mupio_ocu,...,int_o_noint,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve,Núm_corre,zona_ciudad,num_corre
11193,11194.0,2023,12,31,7,0,1,1,2,203,...,2,1,999,99,9999,99,5,NaN,NaN,NaN
11194,11195.0,2023,12,31,7,20,4,3,2,203,...,2,1,999,99,9999,99,5,NaN,NaN,NaN
11195,11196.0,2023,12,31,7,19,4,3,4,401,...,2,11,999,2,9999,99,5,NaN,NaN,NaN
11196,11197.0,2023,12,31,7,17,3,2,3,314,...,1,1,999,2,9999,99,5,NaN,NaN,NaN
11197,11198.0,2023,12,31,7,17,3,2,3,314,...,1,1,999,2,9999,99,5,NaN,NaN,NaN


In [220]:
fallecidos_junto.to_csv('ine/fallecidos_junto.csv', index=False)

### Hechos

In [233]:
columnas = ['núm_corre', 'año_ocu', 'mes_ocu', 'día_ocu', 'hora_ocu','día_sem_ocu','g_hora', 'g_hora_5', 'depto_ocu', 'mupio_ocu', 'área_geo_ocu', 'zona_ocu', 'tipo_veh', 'marca_veh', 'color_veh', 'modelo_veh', 'tipo_eve', 'g_modelo_veh']
hechos15['hora_ocu'] = ''
hechos15 = hechos15[columnas]
hechos20.columns = hechos20.columns.str.lower()

In [234]:
hechos15.columns

Index(['núm_corre', 'año_ocu', 'mes_ocu', 'día_ocu', 'hora_ocu', 'día_sem_ocu',
       'g_hora', 'g_hora_5', 'depto_ocu', 'mupio_ocu', 'área_geo_ocu',
       'zona_ocu', 'tipo_veh', 'marca_veh', 'color_veh', 'modelo_veh',
       'tipo_eve', 'g_modelo_veh'],
      dtype='object')

In [231]:
diccionario_hechos['día_sem_ocu']['Sabado'] = 6
diccionario_hechos['día_sem_ocu']['Miercoles'] = 3
diccionario_hechos['día_sem_ocu']

{'Lunes': 1,
 'Martes': 2,
 'Miércoles': 3,
 'Jueves': 4,
 'Viernes': 5,
 'Sábado': 6,
 'Domingo': 7,
 'Sabado': 6,
 'Miercoles': 3}

In [107]:
diccionario_hechos['g_hora_5']['Ma?ana'] = 1
diccionario_hechos['g_hora_5']

{'Mañana': 1, 'Tarde': 2, 'Noche': 3, 'Ignorada': 4, 'Ma?ana': 1}

In [108]:
dicc_deptos_copia = diccionario_hechos['depto_ocu'].copy()

def tilde_a_signo(texto):
    for c in 'áéíóúÁÉÍÓÚ':
        texto = texto.replace(c, '?')
    return texto

# Crear nuevo diccionario con las llaves "modificadas"
diccionario_hechos['depto_ocu'] = {tilde_a_signo(k): v for k, v in diccionario_hechos['depto_ocu'].items()}

diccionario_hechos['depto_ocu']['Sacatepequez'] = 10
diccionario_hechos['depto_ocu']

{'Guatemala': 1,
 'El Progreso': 2,
 'Sacatep?quez': 3,
 'Chimaltenango': 4,
 'Escuintla': 5,
 'Santa Rosa': 6,
 'Solol?': 7,
 'Totonicap?n': 8,
 'Quetzaltenango': 9,
 'Suchitep?quez': 10,
 'Retalhuleu': 11,
 'San Marcos': 12,
 'Huehuetenango': 13,
 'Quich?': 14,
 'Baja Verapaz': 15,
 'Alta Verapaz': 16,
 'Pet?n': 17,
 'Izabal': 18,
 'Zacapa': 19,
 'Chiquimula': 20,
 'Jalapa': 21,
 'Jutiapa': 22,
 'Sacatepequez': 10}

In [ ]:
for k in list(diccionario_hechos['mupio_ocu'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_hechos['mupio_ocu']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_hechos['mupio_ocu'][nk] = diccionario_hechos['mupio_ocu'].pop(k)

dicc_mupios_copia = diccionario_hechos['mupio_ocu'].copy()

# Crear nuevo diccionario con las llaves "modificadas"
diccionario_hechos['mupio_ocu'] = {tilde_a_signo(k): v for k, v in diccionario_hechos['mupio_ocu'].items()}
diccionario_hechos['mupio_ocu'].update(dicc_mupios_copia)


In [126]:

dicc_vehiculos_copia = diccionario_hechos['tipo_veh'].copy()

# Crear nuevo diccionario con las llaves "modificadas"
diccionario_hechos['tipo_veh'] = {tilde_a_signo(k): v for k, v in diccionario_hechos['tipo_veh'].items()}
diccionario_hechos['tipo_veh'].update(dicc_vehiculos_copia)
diccionario_hechos['tipo_veh']['Moto taxi'] = 14
diccionario_hechos['tipo_veh']

{'Autom?vil': 1,
 'Camioneta': 2,
 'Pick up': 3,
 'Motocicleta': 4,
 'Cami?n': 5,
 'Cabezal': 6,
 'Bus extraurbano': 7,
 'Jeep': 8,
 'Microb?s': 9,
 'Taxi': 10,
 'Panel': 11,
 'Bus urbano': 12,
 'Tractor': 13,
 'Mototaxi': 14,
 'Furg?n': 15,
 'Gr?a': 16,
 'Bus escolar': 17,
 'Bicicleta': 18,
 'Avioneta': 19,
 'Montacargas': 20,
 'Bus militar': 21,
 'Cuatrimoto': 22,
 'Furgoneta': 23,
 'Motos acu?ticas': 24,
 'Ignorado': 99,
 'Automóvil': 1,
 'Camión': 5,
 'Microbús': 9,
 'Furgón': 15,
 'Grúa': 16,
 'Motos acuáticas': 24,
 'Moto taxi': 14}

In [127]:
dicc_vehiculos_copia = diccionario_hechos['tipo_eve'].copy()

# Crear nuevo diccionario con las llaves "modificadas"
diccionario_hechos['tipo_eve'] = {tilde_a_signo(k): v for k, v in diccionario_hechos['tipo_eve'].items()}
diccionario_hechos['tipo_eve'].update(dicc_vehiculos_copia)

diccionario_hechos['tipo_eve']['Embarrancó'] = 7
diccionario_hechos['tipo_eve']

{'Colisi?n': 1,
 'Choque': 2,
 'Vuelco': 3,
 'Ca?da': 4,
 'Atropello': 5,
 'Derrape': 6,
 'Embarranco': 7,
 'Encuneto': 8,
 'Ignorado': 99,
 'Embarranc?': 7,
 'Colisión': 1,
 'Caída': 4,
 'Embarrancó': 7}

In [135]:
for k in list(diccionario_hechos['mes_ocu'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_hechos['mes_ocu']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_hechos['mes_ocu'][nk] = diccionario_hechos['mes_ocu'].pop(k)

diccionario_hechos['mes_ocu']

{'Enero': 1,
 'Febrero': 2,
 'Marzo': 3,
 'Abril': 4,
 'Mayo': 5,
 'Junio': 6,
 'Julio': 7,
 'Septiembre': 9,
 'Octubre': 10,
 'Noviembre': 11,
 'Diciembre': 12,
 'Agosto': 8}

In [136]:
hechos22.replace(diccionario_hechos, inplace=True)
hechos23.replace(diccionario_hechos, inplace=True)

/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_36586/469893416.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hechos22.replace(diccionario_hechos, inplace=True)
/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_36586/469893416.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hechos23.replace(diccionario_hechos, inplace=True)


In [235]:
hechos_junto = pd.concat(lista_hechos)
hechos_junto = hechos_junto[columnas]
hechos_junto.columns

Index(['núm_corre', 'año_ocu', 'mes_ocu', 'día_ocu', 'hora_ocu', 'día_sem_ocu',
       'g_hora', 'g_hora_5', 'depto_ocu', 'mupio_ocu', 'área_geo_ocu',
       'zona_ocu', 'tipo_veh', 'marca_veh', 'color_veh', 'modelo_veh',
       'tipo_eve', 'g_modelo_veh'],
      dtype='object')

In [236]:
hechos_junto.to_csv('ine/hechos_junto.csv', index=False)

25/10/08 22:19:24 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 928994 ms exceeds timeout 120000 ms
25/10/08 22:19:24 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/08 22:19:30 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

### Vehículos

In [187]:
for archivo in lista_vehiculos:

    archivo.columns = archivo.columns.str.lower()

    if 'núm_corre' in archivo.columns:
        archivo.rename(columns={'núm_corre': 'num_corre'}, inplace=True)

In [188]:
vehiculos15.columns

Index(['año_ocu', 'num_corre', 'mes_ocu', 'día_ocu', 'día_sem_ocu', 'hora_ocu',
       'g_hora', 'g_hora_5', 'depto_ocu', 'mupio_ocu', 'área_geo_ocu',
       'zona_ocu', 'sexo_per', 'edad_per', 'mayor_menor', 'g_edad_80ymás',
       'g_edad_60ymás', 'edad_quinquenales', 'estado_con', 'tipo_veh',
       'marca_veh', 'color_veh', 'modelo_veh', 'g_modelo_veh', 'tipo_eve'],
      dtype='object')

In [192]:
diccionario_vehiculos['día_sem_ocu']['Sabado'] = 6
diccionario_vehiculos['día_sem_ocu']['Miercoles'] = 3
diccionario_vehiculos['día_sem_ocu']

{'Lunes': 1,
 'Martes': 2,
 'Miércoles': 3,
 'Jueves': 4,
 'Viernes': 5,
 'Sábado': 6,
 'Domingo': 7,
 'Sabado': 6,
 'Miercoles': 3}

In [194]:
for k in list(diccionario_vehiculos['mupio_ocu'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_vehiculos['mupio_ocu']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_vehiculos['mupio_ocu'][nk] = diccionario_vehiculos['mupio_ocu'].pop(k)

diccionario_vehiculos['mupio_ocu']

{'San José Pinula': 103,
 'San José del Golfo': 104,
 'Palencia': 105,
 'Chinautla': 106,
 'San Pedro Ayampuc': 107,
 'Mixco': 108,
 'San Pedro Sacatepéquez': 1202,
 'San Juan Sacatepéquez': 110,
 'San Raymundo': 111,
 'Chuarrancho': 112,
 'Fraijanes': 113,
 'Amatitlán': 114,
 'Villa Nueva': 115,
 'Villa Canales': 116,
 'Guastatoya': 201,
 'Morazán': 202,
 'San Agustín Acasaguastlán': 203,
 'El Jícaro': 205,
 'Sansare': 206,
 'Sanarate': 207,
 'San Antonio la Paz': 208,
 'Antigua Guatemala': 301,
 'Jocotenango': 302,
 'Pastores': 303,
 'Sumpango': 304,
 'Santo Domingo Xenacoj': 305,
 'Santiago Sacatepéquez': 306,
 'San Bartolomé Milpas Altas': 307,
 'San Lucas Sacatepéquez': 308,
 'Santa Lucía Milpas Altas': 309,
 'Magdalena Milpas Altas': 310,
 'Santa María de Jesús': 311,
 'Ciudad Vieja': 312,
 'San Miguel Dueñas': 313,
 'San Juan Alotenango': 314,
 'San Antonio Aguas Calientes': 315,
 'Santa Catarina Barahona': 316,
 'Chimaltenango': 401,
 'San José  Poaquíl': 402,
 'San Martín Jilo

In [196]:
diccionario_vehiculos['zona_ocu']['Ignorado'] = 99
diccionario_vehiculos['zona_ocu']

{'Ignorada': 99, 'Ignorado': 99}

In [198]:
for k in list(diccionario_vehiculos['edad_quinquenales'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_vehiculos['edad_quinquenales']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_vehiculos['edad_quinquenales'][nk] = diccionario_vehiculos['edad_quinquenales'].pop(k)

diccionario_vehiculos['edad_quinquenales']

{'0 - 4': 1,
 '5 - 9': 2,
 '10 - 14': 3,
 '15 - 19': 4,
 '20 - 24': 5,
 '25 - 29': 6,
 '30 - 34': 7,
 '35 - 39': 8,
 '40 - 44': 9,
 '45 - 49': 10,
 '50 - 54': 11,
 '55 - 59': 12,
 '60 - 64': 13,
 '65 - 69': 14,
 '70 - 74': 15,
 '75 - 79': 16,
 '80 y más': 17,
 'Ignorado': 18}

In [200]:
diccionario_vehiculos['estado_con']['No Ebrio'] = 1
diccionario_vehiculos['estado_con']

{'No ebrio': 1, 'Ebrio': 2, 'Ignorado': 9, 'No Ebrio': 1}

In [207]:
diccionario_vehiculos['tipo_veh']['Moto taxi'] = 14


In [210]:
diccionario_vehiculos['tipo_eve']['Embarrancó'] = 7
diccionario_vehiculos['tipo_eve']['Encunetó'] = 8
diccionario_vehiculos['tipo_eve']

{'Colisión': 1,
 'Choque': 2,
 'Vuelco': 3,
 'Caída': 4,
 'Atropello': 5,
 'Derrape': 6,
 'Embarranco': 7,
 'Encuneto': 8,
 'Ignorado': 99,
 'Embarrancó': 7,
 'Encunetó': 8}

In [215]:
for k in list(diccionario_vehiculos['mes_ocu'].keys()):
    nk = k.strip()
    if nk != k:
        # Si no quieres sobreescribir en caso de colisión:
        if nk in diccionario_vehiculos['mes_ocu']:
            raise KeyError(f"Colisión al limpiar '{k}' → '{nk}' (ya existe).")
        diccionario_vehiculos['mes_ocu'][nk] = diccionario_vehiculos['mes_ocu'].pop(k)

diccionario_vehiculos['mes_ocu']

{'Enero': 1,
 'Febrero': 2,
 'Marzo': 3,
 'Abril': 4,
 'Mayo': 5,
 'Junio': 6,
 'Julio': 7,
 'Septiembre': 9,
 'Octubre': 10,
 'Noviembre': 11,
 'Diciembre': 12,
 'Agosto': 8}

In [216]:
vehiculos22.replace(diccionario_vehiculos, inplace=True)
vehiculos23.replace(diccionario_vehiculos, inplace=True)

/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_36586/4109283465.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vehiculos22.replace(diccionario_vehiculos, inplace=True)
/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_36586/4109283465.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vehiculos23.replace(diccionario_vehiculos, inplace=True)


In [217]:
vehiculos_junto = pd.concat(lista_vehiculos)

In [222]:
vehiculos_junto.to_csv('ine/vehiculos_junto.csv', index=False)